In [1]:
import pandas as pd
from pprint import pprint
import ast
import numpy as np

# match data 전처리

In [60]:
match_df = pd.read_csv('match_data_version1.csv')
match_df.shape

(108829, 15)

### 원본 데이터 확인

In [53]:
list(match_df.columns)

['Unnamed: 0',
 'gameCreation',
 'gameDuration',
 'gameId',
 'gameMode',
 'gameType',
 'gameVersion',
 'mapId',
 'participantIdentities',
 'participants',
 'platformId',
 'queueId',
 'seasonId',
 'status.message',
 'status.status_code']

In [61]:
match_df.head()

,Unnamed: 0,gameCreation,gameDuration,gameId,gameMode,gameType,gameVersion,mapId,participantIdentities,participants,platformId,queueId,seasonId,status.message,status.status_code
0,0,1.585155e+12,1323.0,4.247263e+09,CLASSIC,MATCHED_GAME,10.6.314.4405,11.0,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420.0,13.0,NaN,NaN
1,1,1.585152e+12,1317.0,4.247156e+09,CLASSIC,MATCHED_GAME,10.6.314.4405,11.0,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420.0,13.0,NaN,NaN
2,2,1.585059e+12,932.0,4.243963e+09,CLASSIC,MATCHED_GAME,10.6.313.8894,11.0,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420.0,13.0,NaN,NaN
3,3,1.584978e+12,2098.0,4.241678e+09,CLASSIC,MATCHED_GAME,10.6.313.8894,11.0,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420.0,13.0,NaN,NaN
4,4,1.584973e+12,2344.0,4.241539e+09,CLASSIC,MATCHED_GAME,10.6.313.8894,11.0,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420.0,13.0,NaN,NaN


### 필요없는 컬럼 제외

In [55]:
match_df = match_df.drop(['Unnamed: 0','gameCreation','gameMode','gameType','mapId','platformId','seasonId','status.message','status.status_code'], axis=1)

In [56]:
match_df.head()

,gameDuration,gameId,gameVersion,participantIdentities,participants,queueId
0,1323.0,4.247263e+09,10.6.314.4405,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",420.0
1,1317.0,4.247156e+09,10.6.314.4405,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",420.0
2,932.0,4.243963e+09,10.6.313.8894,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",420.0
3,2098.0,4.241678e+09,10.6.313.8894,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",420.0
4,2344.0,4.241539e+09,10.6.313.8894,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",420.0


### 중복 데이터 제거

In [ ]:
match_df = match_df.drop_duplicates(['gameId'])

### 5v5 랭크게임 및 Reset 게임 제외

In [ ]:
match_df = match_df[(match_df["queueId"] == 420) & (match_df["gameDuration"] > 240)]  # queueId : 420 -> 5v5 Ranked games
                                                                                      # 게임시간 3분( + 게임재시작 투표시간 1분) 이전에 종료된 게임 제외0  

### Dictionary 타입 데이터 확인

In [ ]:
pprint(ast.literal_eval(match_df['participantIdentities'][1]))

In [ ]:
pprint(ast.literal_eval(match_df['participants'][1]))

### nested dictionary를 위한 재귀함수

In [107]:
def solve_dict_column(dict_to_iterate, upper_key=None, pID = None):            # 하드 코딩, 추후 수정 필요
    global column_list
    global value_list
    for key, value in dict_to_iterate.items():
        if key == 'participantId':
            pID = 'P' + str(value)
        
        if type(value) == dict:
            solve_dict_column(value, key, pID)
        else:
            if upper_key == None:
                column_list.append(pID+ '_' + key)

            else :
                column_list.append(pID + '_' + str(upper_key) + '_' + key)
            value_list.append(value)


# 메모리 효율적으로 쓰게 수정 필요

### match data 의 participants, participantIdentities 칼럼의 nested dictionary to Dataframe (string)

In [ ]:
resolved = pd.DataFrame()
n = match_df.shape[0]
except_list = []

for i in range(n):
    temp = pd.DataFrame()
    column_list = []
    value_list = []
    if (len(eval(match_df.iloc[i]['participants'])) ==10) and  (len(eval(match_df.iloc[i]['participantIdentities'])) == 10):
        for x,y in enumerate([0,1,2,5]):
            column_list.append(match_df.columns[y])
            value_list.append(match_df.loc[i][y])
        for j in range(10):
            solve_dict_column(ast.literal_eval(match_df['participants'][i])[j])
            solve_dict_column(ast.literal_eval(match_df['participantIdentities'][i])[j])

        d=[]
        new_d = [] # 중복된 원소만 넣는 리스트
        for k in column_list:
            if k not in d: # 처음 등장한 원소
                d.append(k)
            else:
                if k not in new_d: # 이미 중복 원소로 판정된 경우는 제외
                    new_d.append(k)

        new_d # [1, 2] # 2회 이상 등장한 값들만 담긴 리스트
        
        temp = pd.DataFrame(data=[value_list], columns=column_list)
        temp = temp.drop(new_d, axis=1)

        resolved = pd.concat([resolved, temp], axis=0)
    else:
        except_list.append(i)

    if (i%1000) == 0:
        print(i)


resolved.head()

### Dict 변환 후 필요없는 컬럼 제거

In [ ]:
col_name=resolved.columns.tolist()
col_name

In [ ]:
search = "Diff"
d1=match_df.copy()

removelist=["Diff","player", "Player", "perk", "highestAchievedSeasonTier", "totalScoreRank", "timeline_participantId"]

for i in range(len(removelist)):
    search = removelist[i]

    for word in col_name:
        if search in word: 
            print('>> remove: ' + word)
            d1=d1.drop(word, axis=1)

In [ ]:
d2=d1.drop('Unnamed: 0', axis=1)

In [ ]:
check_list=[]
col_name1 = list(d2.columns)
for i in range(len(col_name1)):
    for j in range(len(d2)):
        if d2.iloc[j,i] == True or False or None:
            if d2.iloc[j,i] == True or False:
                check_list.append(col_name1[i])
                break
        else:
            break
d3 = d2[check_list]

d3.isnull().sum()   ## ??????

In [ ]:
check_list=[]
col_name1 = list(d1.columns)

for i in range(len(col_name1)):
    for j in range(len(d1)):
        if (type(d1.iloc[j,i]) == bool) or (d1.iloc[j,i] == None):
            if (d1.iloc[j,i] == True) or (d1.iloc[j,i] == False):
                check_list.append(col_name1[i])
                break
        else:
            break
d3 = d1[check_list]

d3.isnull().sum()    #### ??????

In [ ]:
del match_df
del final

# winner data 밴픽 dictionary solve

In [83]:
winner_data = pd.read_csv('match_winner_data_version1.csv')
winner_data.shape

(108829, 18)

In [57]:
winner_data.shape[0]

108829

In [8]:
winner_data.head()

,Unnamed: 0,teamId,win,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald,towerKills,inhibitorKills,baronKills,dragonKills,vilemawKills,riftHeraldKills,dominionVictoryScore,bans,gameId
0,0,200,Win,False,True,True,False,True,True,9,1,0,3,0,2,0,"[{'championId': 523, 'pickTurn': 6}, {'champio...",4.247263e+09
1,1,100,Win,False,False,False,False,True,True,4,0,0,2,0,2,0,"[{'championId': 523, 'pickTurn': 1}, {'champio...",4.247156e+09
2,2,200,Win,True,True,True,False,True,True,5,1,0,2,0,2,0,"[{'championId': 350, 'pickTurn': 6}, {'champio...",4.243963e+09
3,3,200,Win,True,True,False,False,False,True,6,0,1,3,0,1,0,"[{'championId': 81, 'pickTurn': 6}, {'champion...",4.241678e+09
4,4,100,Win,True,True,True,True,True,True,11,3,2,2,0,2,0,"[{'championId': 30, 'pickTurn': 1}, {'champion...",4.241539e+09


In [12]:
list(winner_data.columns)

['Unnamed: 0',
 'teamId',
 'win',
 'firstBlood',
 'firstTower',
 'firstInhibitor',
 'firstBaron',
 'firstDragon',
 'firstRiftHerald',
 'towerKills',
 'inhibitorKills',
 'baronKills',
 'dragonKills',
 'vilemawKills',
 'riftHeraldKills',
 'dominionVictoryScore',
 'bans',
 'gameId']

In [50]:
eval(winner_data['bans'][284])

[]

In [26]:
len(winner_data.loc[winner_data['bans'] == "[]"])

24632

In [86]:
for i in range(5):
    winner_data['ban_champion_'+ str(i+1)] = np.NaN

In [87]:
winner_data.head()

,Unnamed: 0,teamId,win,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald,towerKills,...,vilemawKills,riftHeraldKills,dominionVictoryScore,bans,gameId,ban_champion_1,ban_champion_2,ban_champion_3,ban_champion_4,ban_champion_5
0,0,200,Win,False,True,True,False,True,True,9,...,0,2,0,"[{'championId': 523, 'pickTurn': 6}, {'champio...",4.247263e+09,NaN,NaN,NaN,NaN,NaN
1,1,100,Win,False,False,False,False,True,True,4,...,0,2,0,"[{'championId': 523, 'pickTurn': 1}, {'champio...",4.247156e+09,NaN,NaN,NaN,NaN,NaN
2,2,200,Win,True,True,True,False,True,True,5,...,0,2,0,"[{'championId': 350, 'pickTurn': 6}, {'champio...",4.243963e+09,NaN,NaN,NaN,NaN,NaN
3,3,200,Win,True,True,False,False,False,True,6,...,0,1,0,"[{'championId': 81, 'pickTurn': 6}, {'champion...",4.241678e+09,NaN,NaN,NaN,NaN,NaN
4,4,100,Win,True,True,True,True,True,True,11,...,0,2,0,"[{'championId': 30, 'pickTurn': 1}, {'champion...",4.241539e+09,NaN,NaN,NaN,NaN,NaN


In [88]:
n = winner_data.shape[0]

for i in range(n): 
    try:
        for j in range(5):
            for key, value in eval(winner_data['bans'][i])[j].items():
                
                if key == 'championId':
                    winner_data.loc[i,'ban_champion_'+ str(j+1)] = value
    except:
        pass
                
winner_data.head()

,Unnamed: 0,teamId,win,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald,towerKills,...,vilemawKills,riftHeraldKills,dominionVictoryScore,bans,gameId,ban_champion_1,ban_champion_2,ban_champion_3,ban_champion_4,ban_champion_5
0,0,200,Win,False,True,True,False,True,True,9,...,0,2,0,"[{'championId': 523, 'pickTurn': 6}, {'champio...",4.247263e+09,523.0,429.0,142.0,62.0,-1.0
1,1,100,Win,False,False,False,False,True,True,4,...,0,2,0,"[{'championId': 523, 'pickTurn': 1}, {'champio...",4.247156e+09,523.0,429.0,60.0,2.0,53.0
2,2,200,Win,True,True,True,False,True,True,5,...,0,2,0,"[{'championId': 350, 'pickTurn': 6}, {'champio...",4.243963e+09,350.0,4.0,105.0,875.0,80.0
3,3,200,Win,True,True,False,False,False,True,6,...,0,1,0,"[{'championId': 81, 'pickTurn': 6}, {'champion...",4.241678e+09,81.0,110.0,523.0,429.0,60.0
4,4,100,Win,True,True,True,True,True,True,11,...,0,2,0,"[{'championId': 30, 'pickTurn': 1}, {'champion...",4.241539e+09,30.0,429.0,523.0,235.0,7.0


In [91]:
winner_data = winner_data.drop('bans', axis=1)

In [92]:
winner_data.to_csv("winner_edit.csv", mode='w')

In [ ]:
del winner_data

# loser data 밴픽 dictionary solve

In [93]:
loser_data = pd.read_csv('match_loser_data_version1.csv')
loser_data.shape

(108829, 18)

In [95]:
loser_data.shape[0]

108829

In [96]:
loser_data.head()

,Unnamed: 0,teamId,win,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald,towerKills,inhibitorKills,baronKills,dragonKills,vilemawKills,riftHeraldKills,dominionVictoryScore,bans,gameId
0,0,100,Fail,True,False,False,False,False,False,0,0,0,0,0,0,0,"[{'championId': -1, 'pickTurn': 1}, {'champion...",4.247263e+09
1,1,200,Fail,True,True,False,False,False,False,2,0,0,0,0,0,0,"[{'championId': 875, 'pickTurn': 6}, {'champio...",4.247156e+09
2,2,100,Fail,False,False,False,False,False,False,0,0,0,0,0,0,0,"[{'championId': 110, 'pickTurn': 1}, {'champio...",4.243963e+09
3,3,100,Fail,False,False,False,True,True,False,2,0,1,1,0,0,0,"[{'championId': 523, 'pickTurn': 1}, {'champio...",4.241678e+09
4,4,200,Fail,False,False,False,False,False,False,6,1,1,3,0,0,0,"[{'championId': 523, 'pickTurn': 6}, {'champio...",4.241539e+09


In [97]:
list(loser_data.columns)

['Unnamed: 0',
 'teamId',
 'win',
 'firstBlood',
 'firstTower',
 'firstInhibitor',
 'firstBaron',
 'firstDragon',
 'firstRiftHerald',
 'towerKills',
 'inhibitorKills',
 'baronKills',
 'dragonKills',
 'vilemawKills',
 'riftHeraldKills',
 'dominionVictoryScore',
 'bans',
 'gameId']

In [99]:
eval(loser_data['bans'][284])

[]

In [101]:
len(loser_data.loc[loser_data['bans'] == "[]"])

24632

In [103]:
for i in range(5):
    loser_data['ban_champion_'+ str(i+1)] = np.NaN
loser_data.head()

,Unnamed: 0,teamId,win,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald,towerKills,...,vilemawKills,riftHeraldKills,dominionVictoryScore,bans,gameId,ban_champion_1,ban_champion_2,ban_champion_3,ban_champion_4,ban_champion_5
0,0,100,Fail,True,False,False,False,False,False,0,...,0,0,0,"[{'championId': -1, 'pickTurn': 1}, {'champion...",4.247263e+09,NaN,NaN,NaN,NaN,NaN
1,1,200,Fail,True,True,False,False,False,False,2,...,0,0,0,"[{'championId': 875, 'pickTurn': 6}, {'champio...",4.247156e+09,NaN,NaN,NaN,NaN,NaN
2,2,100,Fail,False,False,False,False,False,False,0,...,0,0,0,"[{'championId': 110, 'pickTurn': 1}, {'champio...",4.243963e+09,NaN,NaN,NaN,NaN,NaN
3,3,100,Fail,False,False,False,True,True,False,2,...,0,0,0,"[{'championId': 523, 'pickTurn': 1}, {'champio...",4.241678e+09,NaN,NaN,NaN,NaN,NaN
4,4,200,Fail,False,False,False,False,False,False,6,...,0,0,0,"[{'championId': 523, 'pickTurn': 6}, {'champio...",4.241539e+09,NaN,NaN,NaN,NaN,NaN


In [104]:
n = loser_data.shape[0]

for i in range(n): 
    try:
        for j in range(5):
            for key, value in eval(loser_data['bans'][i])[j].items():
                
                if key == 'championId':
                    loser_data.loc[i,'ban_champion_'+ str(j+1)] = value
    except:
        pass
                
loser_data.head()

,Unnamed: 0,teamId,win,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald,towerKills,...,vilemawKills,riftHeraldKills,dominionVictoryScore,bans,gameId,ban_champion_1,ban_champion_2,ban_champion_3,ban_champion_4,ban_champion_5
0,0,100,Fail,True,False,False,False,False,False,0,...,0,0,0,"[{'championId': -1, 'pickTurn': 1}, {'champion...",4.247263e+09,-1.0,80.0,875.0,555.0,412.0
1,1,200,Fail,True,True,False,False,False,False,2,...,0,0,0,"[{'championId': 875, 'pickTurn': 6}, {'champio...",4.247156e+09,875.0,432.0,76.0,2.0,64.0
2,2,100,Fail,False,False,False,False,False,False,0,...,0,0,0,"[{'championId': 110, 'pickTurn': 1}, {'champio...",4.243963e+09,110.0,142.0,875.0,432.0,80.0
3,3,100,Fail,False,False,False,True,True,False,2,...,0,0,0,"[{'championId': 523, 'pickTurn': 1}, {'champio...",4.241678e+09,523.0,412.0,429.0,80.0,58.0
4,4,200,Fail,False,False,False,False,False,False,6,...,0,0,0,"[{'championId': 523, 'pickTurn': 6}, {'champio...",4.241539e+09,523.0,106.0,80.0,110.0,105.0


In [105]:
loser_data = loser_data.drop('bans', axis=1)

In [106]:
loser_data.to_csv("loser_edit.csv", mode='w')

In [110]:
del loser_data

# champion csv

In [111]:
champion = pd.read_csv('riot_champion.csv')

In [112]:
champion.head

<bound method NDFrame.head of      Unnamed: 0 version       id  key     name                   title  \
0             0  10.6.1   Aatrox  266   Aatrox        the Darkin Blade   
1             0  10.6.1     Ahri  103     Ahri     the Nine-Tailed Fox   
2             0  10.6.1    Akali   84    Akali      the Rogue Assassin   
3             0  10.6.1  Alistar   12  Alistar            the Minotaur   
4             0  10.6.1    Amumu   32    Amumu           the Sad Mummy   
..          ...     ...      ...  ...      ...                     ...   
143           0  10.6.1      Zed  238      Zed   the Master of Shadows   
144           0  10.6.1    Ziggs  115    Ziggs  the Hexplosives Expert   
145           0  10.6.1   Zilean   26   Zilean        the Chronokeeper   
146           0  10.6.1      Zoe  142      Zoe  the Aspect of Twilight   
147           0  10.6.1     Zyra  143     Zyra      Rise of the Thorns   

                                                 blurb                  tags  \
0

In [113]:
list(champion.columns)

['Unnamed: 0',
 'version',
 'id',
 'key',
 'name',
 'title',
 'blurb',
 'tags',
 'partype',
 'info.attack',
 'info.defense',
 'info.magic',
 'info.difficulty',
 'image.full',
 'image.sprite',
 'image.group',
 'image.x',
 'image.y',
 'image.w',
 'image.h',
 'stats.hp',
 'stats.hpperlevel',
 'stats.mp',
 'stats.mpperlevel',
 'stats.movespeed',
 'stats.armor',
 'stats.armorperlevel',
 'stats.spellblock',
 'stats.spellblockperlevel',
 'stats.attackrange',
 'stats.hpregen',
 'stats.hpregenperlevel',
 'stats.mpregen',
 'stats.mpregenperlevel',
 'stats.crit',
 'stats.critperlevel',
 'stats.attackdamage',
 'stats.attackdamageperlevel',
 'stats.attackspeedperlevel',
 'stats.attackspeed']

In [116]:
champion = champion.iloc[:,[2,3,7]]
champion.head() 

,id,key,tags
0,Aatrox,266,"['Fighter', 'Tank']"
1,Ahri,103,"['Mage', 'Assassin']"
2,Akali,84,['Assassin']
3,Alistar,12,"['Tank', 'Support']"
4,Amumu,32,"['Tank', 'Mage']"


In [117]:
champion.to_csv("champion_edit.csv", mode='w')

In [118]:
del champion

In [ ]:
test

test